## 迁移学习

训练参数设置

In [3]:
#pre_features_files = {'ResNet50':'ResNet50_pre_out.h5', 'Xception':'Xception_pre_out.h5', 'InceptionV3':'InceptionV3_pre_out.h5'}
pre_features_files = {'ResNet50':'ResNet50_pre_out.h5', 
                      'Xception':'Xception_pre_out.h5', 
                      'InceptionResNetV2':'InceptionResNetV2_pre_out.h5'}

composite_model = False
single_model = 'InceptionResNetV2'

# check training parameter
if not composite_model and single_model not in pre_features_files:
    print("%s not found in pre-trained models" %single_model)
else:
    if composite_model:
        print("will use composite model")
    else:
        print("will use %s model" %single_model)
        
if composite_model:
    model_name = 'Composite_Model'
else:
    model_name = single_model
    
print('model name:', model_name)

will use InceptionResNetV2 model
model name: InceptionResNetV2


### Tools

In [4]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

def show_history(hist, title_name):
    # epoch as x-axis 
    epochs = range(len(hist['acc']))
    
    fig, ax1 = plt.subplots()
    
    ## left scale: acc, val_acc
    # acc
    line_acc = ax1.plot(epochs, hist['acc'], 'r', label='acc')
    # loss
#     ax1.plot(epochs, hist['loss'], 'g', label='loss')
    # val_acc
    line_val_acc = ax1.plot(epochs, hist['val_acc'], 'b', label='val_acc')
    # val_loss
#     ax1.plot(epochs, hist['val_loss'], 'k', label='val_loss')
    
    ax1.set(xlabel='epochs', ylabel='acc',title=title_name)
    
    ## right scale: loss, val_loss
    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
    color = 'tab:blue'
    ax2.set_ylabel('loss', color=color)  # we already handled the x-label with ax1
    line_loss = ax2.plot(epochs, hist['loss'], 'g', label='loss')
    line_val_loss = ax2.plot(epochs, hist['val_loss'], 'k', label='val_loss')
    ax2.tick_params(axis='y', labelcolor=color)

#     ax1.legend(loc='upper right', fancybox=True,)
#     ax2.legend(loc='lower right', fancybox=True)
    
    ax1.grid(True)
    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    
    acc_patch = mpatches.Patch(color='r', label='acc',linestyle='-')
    val_acc_patch = mpatches.Patch(color='b', label='val-acc')
    loss_patch = mpatches.Patch(color='g', label='loss')
    val_loss_patch = mpatches.Patch(color='k', label='val-loss')
    fig.legend(handles=[acc_patch, val_acc_patch, loss_patch, val_loss_patch],
               ncol=4, loc='lower center',
               mode="expand", borderaxespad=0.)

    fig.show()

### 载入数据

In [16]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(66)

train_data = []
test_data = []

# #------- single pre-trained model
if not composite_model:
    with h5py.File(pre_features_files[single_model] , 'r') as h:
        train_data.append(np.array(h['train']))
        train_labels = np.array(h['label'])
        test_data.append(np.array(h['test']))
else:#-------- composite model
    for modelname in pre_features_files:
        with h5py.File(pre_features_files[modelname]) as h:
            train_data.append(np.array(h['train']))
            train_labels = np.array(h['label'])
            test_data.append(np.array(h['test']))

train_data = np.concatenate(train_data, axis=1)
test_data = np.concatenate(test_data, axis=1)

print("train shape:",train_data.shape)
print("train_data:",train_data[:5])
# 预存的X_train, y_train是按顺序存放的，前12500是猫，后12500是狗, 这里打乱顺序，使之随机存放
# Note: 打乱的是存放存放顺序，并不改变 X_train , y_train的对应关系
train_data, train_labels = shuffle(train_data, train_labels)

train_data_unclean = train_data
train_labels_unclean = train_labels

train shape: (25000, 1536)
train_data: [[0.20157795 0.33358407 0.09585248 ... 0.805622   1.0074369  0.4330498 ]
 [0.10468061 0.00802733 0.02267138 ... 0.7481499  0.33362135 0.0298912 ]
 [0.09410941 0.06788639 0.11360098 ... 0.90750456 0.28331906 0.42946684]
 [0.20197937 0.02258976 0.04261705 ... 0.66384625 0.09021231 0.07666409]
 [0.29612273 0.52252316 0.063084   ... 0.77111566 0.33219925 0.24683997]]


#### 剔除train中的异常值

In [6]:
## outliers list
train_outliers=['cat.10636.jpg', 'dog.10161.jpg', 'dog.2614.jpg', 'cat.7564.jpg', 'cat.7377.jpg', 'cat.3216.jpg', 'dog.2422.jpg', 'dog.11266.jpg', 'cat.5071.jpg', 'cat.6345.jpg', 'dog.1773.jpg', 'cat.8456.jpg', 'dog.10190.jpg', 'cat.8921.jpg', 'cat.10029.jpg', 'cat.3004.jpg', 'cat.9171.jpg', 'dog.4218.jpg', 'cat.5418.jpg', 'cat.5974.jpg', 'dog.1895.jpg', 'cat.2520.jpg', 'cat.9983.jpg', 'dog.4507.jpg', 'cat.4338.jpg', 'dog.9188.jpg', 'dog.9517.jpg', 'cat.4308.jpg', 'dog.6475.jpg', 'cat.10365.jpg', 'dog.10237.jpg', 'cat.8470.jpg', 'dog.1625.jpg', 'dog.4367.jpg', 'dog.11299.jpg', 'cat.10536.jpg', 'cat.5351.jpg', 'dog.5604.jpg', 'dog.10747.jpg', 'cat.7464.jpg', 'dog.1259.jpg', 'cat.4688.jpg', 'cat.12272.jpg', 'dog.3889.jpg', 'dog.8898.jpg', 'dog.7706.jpg', 'cat.10712.jpg', 'cat.11184.jpg', 'cat.12476.jpg', 'cat.7968.jpg', 'dog.8736.jpg', 'dog.6725.jpg', 'cat.1139.jpg', 'cat.6348.jpg', 'dog.12376.jpg', 'dog.2339.jpg', 'dog.11437.jpg', 'dog.10801.jpg', 'cat.12424.jpg', 'dog.1194.jpg', 'dog.7076.jpg', 'cat.11039.jpg', 'cat.3672.jpg', 'cat.2939.jpg']

train_outliers.remove('cat.1139.jpg')
train_outliers.remove('cat.3004.jpg')
train_outliers.remove('dog.7706.jpg')

print(len(train_outliers))

61


In [17]:
## create train_generator
# import pandas as pd
from keras.preprocessing.image import *

gen = ImageDataGenerator()
train_generator = gen.flow_from_directory("pre-train", (224, 224), shuffle=False, 
                                         batch_size=16, class_mode=None)

Found 25000 images belonging to 2 classes.


In [19]:

outliers_index = [i for i,fn in enumerate(train_generator.filenames) if fn.split('/')[1] in train_outliers]
train_data = np.array([x for i,x in enumerate(train_data_unclean) if i not in outliers_index])
train_data.shape

(24939, 1536)

In [20]:
train_labels = np.array([x for i,x in enumerate(train_labels_unclean) if i not in outliers_index])
train_labels.shape

(24939,)

### 构建模型

In [26]:
from keras.models import Sequential
from keras.models import Model
from keras.layers import GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input
import numpy as np

np.random.seed(666)

input_tensor = Input(train_data.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

### 在pre-train features上训练模型

In [27]:
from keras.callbacks import EarlyStopping

fit_callback = [EarlyStopping(monitor='val_loss',patience=4)]

model_hist = model.fit(train_data, train_labels,
                       batch_size=128,epochs=20,
                       validation_split = 0.2,
                       verbose=1,
                       callbacks=fit_callback
          )

show_history(model_hist.history, model_name)

Train on 19951 samples, validate on 4988 samples
Epoch 1/20
19951/19951 [==============================] - 1s 65us/step - loss: 0.1426 - acc: 0.9655 - val_loss: 0.0456 - val_acc: 0.9904
Epoch 2/20
19951/19951 [==============================] - 1s 68us/step - loss: 0.0385 - acc: 0.9919 - val_loss: 0.0295 - val_acc: 0.9926
Epoch 3/20
19951/19951 [==============================] - 1s 65us/step - loss: 0.0298 - acc: 0.9933 - val_loss: 0.0238 - val_acc: 0.9942
Epoch 4/20
19951/19951 [==============================] - 1s 64us/step - loss: 0.0245 - acc: 0.9941 - val_loss: 0.0225 - val_acc: 0.9940
Epoch 5/20
19951/19951 [==============================] - 1s 75us/step - loss: 0.0238 - acc: 0.9938 - val_loss: 0.0211 - val_acc: 0.9950
Epoch 6/20
19951/19951 [==============================] - 2s 98us/step - loss: 0.0209 - acc: 0.9946 - val_loss: 0.0202 - val_acc: 0.9954
Epoch 7/20
19951/19951 [==============================] - 2s 109us/step - loss: 0.0199 - acc: 0.9947 - val_loss: 0.0231 - val_acc

TypeError: legend() missing 1 required positional argument: 'labels'

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f729e91b048> (for post_execute):


ValueError: to_rgba: Invalid rgba arg "tab:blue"
to_rgb: Invalid rgb arg "tab:blue"
could not convert string to float: 'tab:blue'

ValueError: to_rgba: Invalid rgba arg "tab:blue"
to_rgb: Invalid rgb arg "tab:blue"
could not convert string to float: 'tab:blue'

In [23]:
logpath = 'log/'+model_name
    
model.save(logpath+'.h5')
with open(logpath+'_hist.txt','w') as f:
    f.write(str(model_hist.history))

In [24]:
## 查看 history
# Pretty display for notebooks
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches


def read_history_from_file(filename):
    f = open(filename,'r')
    rdbuf = f.read()
    hist = eval(rdbuf)
    f.close()
    
    return hist
    
histfile = 'log/'+model_name+'_hist.txt'
hist = read_history_from_file(histfile)
print(hist)

title_name = model_name
show_history(hist, title_name)

{'val_loss': [0.04558143368903503, 0.029494310543239834, 0.023805531762008583, 0.022491690866296637, 0.02110762497575979, 0.02024809735936935, 0.023099472925497133, 0.021064558074643063, 0.019844631612922253, 0.019582695077504268, 0.01905126494759546, 0.019430680959257682, 0.019731520344267967, 0.018601882711363583, 0.0190812131582663, 0.01838371634734279, 0.018692744066811793, 0.01900667449236872, 0.02006441438398843, 0.018924770043280092], 'loss': [0.1426204099704476, 0.03853831470048515, 0.029821116278018697, 0.02451074449007036, 0.023777611145261317, 0.020897374018013976, 0.019933819738696807, 0.020544571566645343, 0.020211097042960743, 0.019508307728835928, 0.02000160874471617, 0.018953283522641244, 0.019366588367370124, 0.019538514723882815, 0.018543323072950785, 0.01905619970106758, 0.018528523259034795, 0.017703767366975084, 0.01839965689155781, 0.017501720666027763], 'acc': [0.9654653902060047, 0.9919302290612, 0.9933336674853391, 0.9940855095281462, 0.9937847726930981, 0.9946

TypeError: legend() missing 1 required positional argument: 'labels'

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f729e91b048> (for post_execute):


ValueError: to_rgba: Invalid rgba arg "tab:blue"
to_rgb: Invalid rgb arg "tab:blue"
could not convert string to float: 'tab:blue'

ValueError: to_rgba: Invalid rgba arg "tab:blue"
to_rgb: Invalid rgb arg "tab:blue"
could not convert string to float: 'tab:blue'

### 用拟合后的模型预测

In [8]:
y_pred = model.predict(test_data, verbose=1)

12500/12500 [==============================] - 0s 23us/step


预测结果写入kaggle sample_submission.csv

In [9]:
y_pred = y_pred.clip(min=0.005, max=0.995)

import pandas as pd
from keras.preprocessing.image import *

df = pd.read_csv("sample_submission.csv")

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("pre-test", (224, 224), shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv(model_name+'_pred.csv', index=None)
df.head(10)

Found 12500 images belonging to 1 classes.


/home/jerry/.local/lib/python3.5/site-packages/ipykernel_launcher.py:14: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


,id,label
0,1,0.995
1,2,0.995
2,3,0.995
3,4,0.995
4,5,0.005
5,6,0.005
6,7,0.005
7,8,0.005
8,9,0.005
9,10,0.005


### 模型可视化

In [1]:
from graphviz import Digraph

dot = Digraph(comment='The Model Chart')

dot.attr('node',shape='record')

dot.node('IN', 'Input|images')
dot.node('PRE-R', 'ResNet50|{input:|output:}|{(224, 224, 3)|(2048)}')
dot.node('PRE-X', 'Xception|{input:|output:}|{(299, 299, 3)|(2048)}')
dot.node('PRE-I', 'InceptionV3|{input:|output:}|{(299, 299, 3)|(2048)}')
dot.edge('IN', 'PRE-R')
dot.edge('IN', 'PRE-X')
dot.edge('IN', 'PRE-I')
# dot.edges(['AB', 'AL'])
# dot.edge('PRE-R', 'PRE-X', constraint='false')

dot.node('L3', 'Flatten|{input:|output:}|{(2048,3)|2048*3=6144}')
dot.edge('PRE-R', 'L3')
dot.edge('PRE-X', 'L3')
dot.edge('PRE-I', 'L3')

dot.node('L4', 'Dropout|Rate:|0.5')
dot.node('L5', 'Output|{input:|output:}|{6144|1}')
dot.edge('L3', 'L4')
dot.edge('L4', 'L5')

print(dot.source)
dot.render('model-table.gv', view=True)

ImportError: No module named 'graphviz'